In [1]:
import pickle
import os
import sys
import pandas as pd
import numpy as np
import re

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model ,load_model


from lifelines import CoxPHFitter
from Models.RCFR_AC_NOCL import SetModel
from Module.DataProcessing import DataLoad
from Module.MetricsGroupNOCL import DoMetric, DoAggMetric, DoSimEval



os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"



### Data load

In [2]:
StackedData, IntToGene, TTE, EVENT, TrIndEmbeddMask, ReferencePatIDLong, ReferencePatIDShort, NormDismInd, MergedData= DataLoad()

PatIDX = StackedData[:, 0:1].astype('int')
GeneIDX = StackedData[:, 1:2].astype('int')
GeneExp = StackedData[:, 2:3]

IndN = len(np.unique(PatIDX))
FeatN = len(np.unique(GeneIDX))


## Setting for tasks
#### WeightID: W0 = for prognosis-positive gene set, W20 = for prognosis-negative gene set

In [3]:
# Parameters for post-hoc models
ModelID ='M04'
WeightID = 'W0'

EmbedSize = 50
NCL_Feat = 5
NCL_Ind = 2
AdjCosWeight_ = 1. # This parameter affects only training phases, so thus any float number can be set in the post analysis phases. 
NumGene_CL = 100

ModelList = os.listdir('./Results/')
ModelList = [i for i in ModelList if ModelID in i and WeightID  in i  and 'hdf5' in i and 'ACNOCL' in i]
FilePath = './Results/'

# Model structure load
RCFR_AC, LayerList = SetModel(AdjCosWeight_, NormDismInd, TrIndEmbeddMask, IndN, FeatN, ReferencePatIDLong, ReferencePatIDShort)

# Data for calculating metric
DataMetric = [MergedData, TTE, EVENT, NCL_Ind,  NumGene_CL, IntToGene]

ColList = ['Model','AvgtPRate', 'AvgtAdjPRate', 'MintAdjPRate', 'AvgABSGeCohD', 'MinABSGeCohD', 'AvgABSSurvCoef', 'MinABSSurvCoef', 'AvgSurvpVal', 
           'MaxSurvpVal', 'NegExpAvgSurvpVal', 'NegExpMinSurvpVal', 'AvgNegSigRate',  'MinNegSigRate', 'AvgPosSigRate', 'MinPosSigRate','IndCentRatio']


500

## Procedure for Selecting best model

In [10]:
MetricTable = pd.DataFrame(columns=ColList)
InfoFeatGroupList = []

for num, model in enumerate(ModelList[:]):
    print(num)
    

    RCFR_AC.load_weights(FilePath + model)  # Model weights load
    InpInd, InpFeat, IndEmbeddWeig, IndEmbeddReferenceLong, FeatEmbeddWeig, IndCentroid, ICosCLSim = LayerList
        
    # Metric calculation: InfoFeatGroup will be used in UMAP analysis
    metrics, InfoFeatGroup = DoMetric (DataMetric, [InpInd, InpFeat, IndEmbeddWeig, FeatEmbeddWeig, IndCentroid,  ICosCLSim])
    InfoFeatGroupList.append(InfoFeatGroup)
    print(InfoFeatGroup)
    MetricTable = pd.concat([MetricTable, pd.DataFrame([[model] + metrics], columns=ColList)], axis=0)
    
    
MetricTable['GroupM'] = np.array([re.findall('.\d+', i)[1][1:] for i in  MetricTable['Model']])
MetricTable['EpNum'] = np.array([ re.findall('.\d+\.', i)[0][1:-1] for i in  MetricTable['Model']]).astype('int')
MetricTable = MetricTable.sort_values(['GroupM','EpNum'])

MetricTable.to_csv('./MetricTables/RCFR_ACNOCLMetricTable_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv',index=False)

0
0
0
0
0
[array([0.01, 0.  , 0.  , 0.  ]), array([0., 0., 0., 0.]), [0.448, 0.559, 0.822, 0.576]]
1
0
0
0
0
[array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), [0.769, 0.771, 0.393, 0.811]]
2
0
0
0
0
[array([0.27, 0.81, 0.03, 0.2 ]), array([0.  , 0.  , 0.  , 0.02]), [0.684, 0.143, 0.174, 0.196]]
3
0
0
0
0
[array([0.39, 0.9 , 0.  , 0.52]), array([0.  , 0.01, 0.  , 0.05]), [0.0, 0.0, 0.0, 0.0]]
4
0
0
0
0
[array([0.11, 0.93, 0.08, 0.14]), array([0.  , 0.  , 0.  , 0.09]), [0.0, 0.0, 0.0, 0.0]]
5
0
0
0
0
[array([0.06, 0.82, 0.07, 0.2 ]), array([0.  , 0.  , 0.  , 0.18]), [0.0, 0.0, 0.0, 0.0]]
6
0
0
0
0
[array([0.06, 0.78, 0.1 , 0.17]), array([0.  , 0.  , 0.03, 0.18]), [0.0, 0.0, 0.0, 0.0]]
7
0
0
0
0
[array([0.02, 0.75, 0.17, 0.18]), array([0.  , 0.  , 0.01, 0.2 ]), [0.0, 0.0, 0.0, 0.0]]
8
0
0
0
0
[array([0.  , 0.82, 0.06, 0.2 ]), array([0.  , 0.  , 0.02, 0.11]), [0.0, 0.0, 0.0, 0.0]]
9
0
0
0
0
[array([0.07, 0.88, 0.02, 0.12]), array([0.  , 0.  , 0.  , 0.16]), [0.0, 0.0, 0.0, 0.0]]
10
0
0
0
0

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1666: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1720: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1722: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1727: RuntimeWarning: divide by zero encountered in log
  log_lik = log_lik + dot(x_death_sum, beta) + weighted_average * log(denom).sum()
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1714: RuntimeWarning: invalid value encountered in multiply
  denom = 1.0 / (risk_phi - increasing_proportion * tie_phi)
c:\users\user\anaconda3\envs\ai\lib\site-package

0
[array([0.64, 0.98, 0.37, 0.01]), array([0.  , 0.  , 0.  , 0.66]), [0.001, 0.005, 0.0, 1.0]]
26
0
0
0
0
[array([0.83, 0.98, 0.65, 0.64]), array([0.  , 0.01, 0.  , 0.02]), [0.001, 0.048, 0.0, 0.0]]
27
0
0
0
0
[array([0.68, 0.99, 0.57, 0.7 ]), array([0.  , 0.  , 0.  , 0.03]), [0.0, 0.004, 0.0, 0.0]]
28
0
0
0
0
[array([0.57, 0.98, 0.43, 0.  ]), array([0.  , 0.  , 0.01, 0.72]), [0.001, 0.009, 0.0, 1.0]]
29
0
0
0
0
[array([0.45, 0.95, 0.52, 0.62]), array([0.  , 0.01, 0.  , 0.03]), [0.001, 0.008, 0.0, 0.0]]
30
0
0
0
0
[array([0.56, 0.94, 0.24, 0.52]), array([0., 0., 0., 0.]), [0.0, 0.006, 0.0, 0.0]]
31
0
0
0
0
[array([0.64, 0.97, 0.25, 0.57]), array([0.  , 0.  , 0.  , 0.01]), [0.0, 0.006, 0.0, 0.0]]
32
0
0
0
0
[array([0.35, 0.99, 0.06, 0.45]), array([0.  , 0.  , 0.03, 0.07]), [0.0, 0.0, 0.0, 0.0]]
33
0
0
0
0
[array([0.16, 0.99, 0.17, 0.24]), array([0.  , 0.  , 0.01, 0.01]), [0.0, 0.0, 0.0, 0.0]]
34
0
0
0
0
[array([0.18, 0.98, 0.11, 0.32]), array([0.  , 0.  , 0.01, 0.04]), [0.0, 0.0, 0.0, 0

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1601: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
  warnings.warn(


0
0
[array([0.14, 0.  , 0.02, 0.77]), array([0.01, 0.99, 0.  , 0.02]), [0.98, 0.741, 0.711, 0.064]]
44
0
0
0
0
[array([0.01, 0.  , 0.  , 0.04]), array([0.1 , 0.92, 0.  , 0.46]), [0.174, 0.03, 0.0, 0.029]]
45
0
0
0
0
[array([0.03, 0.  , 0.  , 0.06]), array([0.17, 0.94, 0.  , 0.66]), [0.004, 0.159, 0.0, 0.0]]
46
0
0
0
0
[array([0.  , 0.  , 0.  , 0.14]), array([0.01, 0.69, 0.  , 0.38]), [0.245, 0.0, 0.0, 0.0]]
47
0
0
0
0
[array([0.  , 0.  , 0.  , 0.08]), array([0.19, 0.69, 0.06, 0.42]), [0.002, 0.0, 0.0, 0.0]]
48
0
0
0
0
[array([0.01, 0.  , 0.  , 0.12]), array([0.05, 0.  , 0.1 , 0.36]), [0.002, 0.0, 0.0, 0.0]]
49
0
0
0
0
[array([0.01, 0.  , 0.  , 0.08]), array([0.07, 0.01, 0.26, 0.5 ]), [0.0, 0.0, 0.0, 0.0]]
50
0
0
0
0
[array([0.  , 0.  , 0.  , 0.05]), array([0.05, 0.07, 0.15, 0.27]), [0.0, 0.0, 0.0, 0.0]]
51
0
0
0
0
[array([0.  , 0.  , 0.  , 0.09]), array([0.05, 0.07, 0.25, 0.46]), [0.0, 0.0, 0.0, 0.0]]
52
0
0
0
0
[array([0.  , 0.  , 0.  , 0.07]), array([0.12, 0.01, 0.24, 0.53]), [0.0, 0

...

0
[array([0.67, 0.98, 0.57, 0.59]), array([0.  , 0.01, 0.  , 0.  ]), [0.001, 0.009, 0.0, 0.0]]
429
0
0
0
0
[array([0.53, 1.  , 0.45, 0.59]), array([0.  , 0.  , 0.  , 0.04]), [0.001, 0.008, 0.0, 0.0]]
430
0
0
0
0
[array([0.5 , 0.97, 0.34, 0.58]), array([0.  , 0.01, 0.  , 0.01]), [0.0, 0.001, 0.0, 0.0]]
431
0
0
0
0
[array([0.59, 0.92, 0.25, 0.51]), array([0.  , 0.  , 0.01, 0.01]), [0.0, 0.006, 0.0, 0.0]]
432
0
0
0
0
[array([0.38, 0.97, 0.04, 0.32]), array([0.01, 0.  , 0.  , 0.07]), [0.0, 0.0, 0.0, 0.0]]
433
0
0
0
0
[array([0.21, 0.96, 0.08, 0.2 ]), array([0.  , 0.  , 0.  , 0.05]), [0.0, 0.0, 0.0, 0.0]]
434
0
0
0
0
[array([0.1 , 0.99, 0.12, 0.33]), array([0.  , 0.  , 0.01, 0.03]), [0.0, 0.0, 0.0, 0.0]]
435
0
0
0
0
[array([0.4 , 0.97, 0.16, 0.57]), array([0.  , 0.  , 0.  , 0.05]), [0.0, 0.049, 0.0, 0.0]]
436
0
0
0
0
[array([0.18, 0.96, 0.06, 0.34]), array([0.  , 0.  , 0.04, 0.01]), [0.0, 0.0, 0.0, 0.0]]
437
0
0
0
0
[array([0.23, 0.98, 0.15, 0.43]), array([0.  , 0.  , 0.02, 0.02]), [0.0, 0.

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1666: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1720: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1722: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1727: RuntimeWarning: divide by zero encountered in log
  log_lik = log_lik + dot(x_death_sum, beta) + weighted_average * log(denom).sum()
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1714: RuntimeWarning: invalid value encountered in multiply
  denom = 1.0 / (risk_phi - increasing_proportion * tie_phi)
c:\users\user\anaconda3\envs\ai\lib\site-package

0
0
[array([0.24, 0.  , 0.  , 0.73]), array([0.04, 0.95, 0.  , 0.02]), [0.893, 0.741, 1.0, 0.064]]
444
0
0
0
0
[array([0.  , 0.  , 0.  , 0.06]), array([0.09, 0.89, 0.  , 0.57]), [0.129, 0.03, 0.0, 0.029]]
445
0
0
0
0
[array([0.01, 0.  , 0.  , 0.04]), array([0.16, 0.91, 0.  , 0.62]), [0.019, 0.276, 0.0, 0.0]]
446
0
0
0
0
[array([0.  , 0.  , 0.  , 0.04]), array([0.  , 0.75, 0.  , 0.39]), [0.244, 0.0, 0.0, 0.0]]
447
0
0
0
0
[array([0.  , 0.  , 0.  , 0.08]), array([0.04, 0.77, 0.25, 0.49]), [0.002, 0.0, 0.0, 0.0]]
448
0
0
0
0
[array([0.01, 0.  , 0.  , 0.09]), array([0.01, 0.  , 0.01, 0.46]), [0.002, 0.0, 0.0, 0.0]]
449
0
0
0
0
[array([0.01, 0.  , 0.  , 0.07]), array([0.06, 0.  , 0.09, 0.4 ]), [0.0, 0.0, 0.0, 0.0]]
450
0
0
0
0
[array([0.  , 0.  , 0.  , 0.09]), array([0.05, 0.04, 0.2 , 0.41]), [0.0, 0.0, 0.0, 0.0]]
451
0
0
0
0
[array([0.  , 0.  , 0.  , 0.11]), array([0.14, 0.09, 0.27, 0.42]), [0.0, 0.0, 0.0, 0.0]]
452
0
0
0
0
[array([0.  , 0.  , 0.  , 0.12]), array([0.2 , 0.03, 0.24, 0.61]),

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1666: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1718: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / np.array([risk_phi])
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1720: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1722: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1714: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / (risk_phi - increasing_proportion * tie_phi)
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.p

0
0
[array([0.57, 0.55, 0.  , 0.68]), array([0.  , 0.  , 0.  , 0.02]), [0.0, 0.005, 1.0, 0.0]]
497
0
0
0
0
[array([0.34, 0.11, 0.  , 0.24]), array([0.  , 0.  , 0.  , 0.03]), [0.018, 0.0, 0.0, 0.0]]
498
0
0
0
0
[array([0.47, 0.78, 0.  , 0.37]), array([0.  , 0.  , 0.  , 0.01]), [0.0, 0.0, 0.0, 0.0]]
499
0
0
0
0
[array([0.34, 0.46, 0.  , 0.08]), array([0.  , 0.  , 0.  , 0.09]), [0.0, 0.0, 0.0, 0.0]]


## Priority-based filter out by metrics

In [4]:
def Aggregation(MetricTable,AggMetricList):
    AggMetricTable = DoSimEval(MetricTable, 'MaxSurvpVal',pCutoff, AggMetricList, ExcRate, NmodEahG)
    AggMetricRank = DoAggMetric(AggMetricList, AggMetricTable[['Model']+AggMetricList]).sort_values('Metrics')
    AggMetricRank = pd.merge(AggMetricRank, AggMetricTable[['Model','MaxSurvpVal']], on='Model', how='left')
    BestModel = AggMetricRank.sort_values('Metrics').iloc[-1]
    
    return AggMetricRank, BestModel

In [5]:
pCutoff = 0.005
ExcRate = 0.2
NmodEahG = 1

NegMetricList = ['IndCentRatio', 'MinABSSurvCoef', 'AvgABSSurvCoef',  'MinNegSigRate', 'AvgNegSigRate', 'MinABSGeCohD', 'AvgABSGeCohD']
PosMetricList = ['IndCentRatio', 'MinABSSurvCoef', 'AvgABSSurvCoef', 'MinPosSigRate', 'AvgPosSigRate', 'MinABSGeCohD', 'AvgABSGeCohD']

MetricTable = pd.read_csv('./MetricTables/RCFR_ACNOCLMetricTable_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv')
MetricTable['GroupM'] = np.array([re.findall('.\d+', i)[2][1:] for i in  MetricTable['Model']])
MetricTable['EpNum'] = np.array([ re.findall('.\d+\.', i)[0][1:-1] for i in  MetricTable['Model']]).astype('int')




In [6]:
NegAggMetricRank, NegBestModel =  Aggregation(MetricTable, NegMetricList)
PosAggMetricRank, PosBestModel =  Aggregation(MetricTable, PosMetricList)

NegAggMetricRank.to_csv('./MetricTables/RCFR_ACNOCL_Neg_AggMetricRank_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv',index=False)
PosAggMetricRank.to_csv('./MetricTables/RCFR_ACNOCL_Pos_AggMetricRank_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv',index=False)

1
N obs with filter of IndCentRatio : 52
N obs with filter of MinABSSurvCoef : 42
N obs with filter of AvgABSSurvCoef : 34
N obs with filter of MinNegSigRate : 28
N obs with filter of AvgNegSigRate : 23
N obs with filter of MinABSGeCohD : 19
N obs with filter of AvgABSGeCohD : 16

2
N obs with filter of IndCentRatio : 54
N obs with filter of MinABSSurvCoef : 44
N obs with filter of AvgABSSurvCoef : 36
N obs with filter of MinNegSigRate : 29
N obs with filter of AvgNegSigRate : 24
N obs with filter of MinABSGeCohD : 20
N obs with filter of AvgABSGeCohD : 16

3
N obs with filter of IndCentRatio : 52
N obs with filter of MinABSSurvCoef : 42
N obs with filter of AvgABSSurvCoef : 34
N obs with filter of MinNegSigRate : 28
N obs with filter of AvgNegSigRate : 23
N obs with filter of MinABSGeCohD : 19
N obs with filter of AvgABSGeCohD : 16

4
N obs with filter of IndCentRatio : 55
N obs with filter of MinABSSurvCoef : 44
N obs with filter of AvgABSSurvCoef : 36
N obs with filter of MinNegSigR

In [14]:
NegAggMetricRank[NegMetricList]

,IndCentRatio,MinABSSurvCoef,AvgABSSurvCoef,MinNegSigRate,AvgNegSigRate,MinABSGeCohD,AvgABSGeCohD
0,0.413093,2.735418,3.618591,0.12,0.3400,0.098562,0.198620
1,0.419501,2.675531,3.603483,0.16,0.3450,0.094727,0.190321
2,0.454545,2.078099,3.319137,0.10,0.4000,0.105341,0.206461
3,0.403587,2.076625,3.064633,0.16,0.4125,0.109601,0.224384
4,0.441941,2.048776,3.461061,0.19,0.4150,0.104647,0.205976


In [15]:
PosAggMetricRank[PosMetricList]

,IndCentRatio,MinABSSurvCoef,AvgABSSurvCoef,MinPosSigRate,AvgPosSigRate,MinABSGeCohD,AvgABSGeCohD
0,0.372093,2.133994,3.351779,0.19,0.3625,0.111486,0.177660
1,0.372093,2.133994,3.358863,0.22,0.4175,0.116090,0.187152
2,0.372093,2.133994,3.333053,0.37,0.4525,0.116775,0.190120
3,0.372093,2.133994,3.341116,0.26,0.4250,0.126021,0.192215
4,0.376093,2.146209,3.349088,0.26,0.4175,0.119659,0.184884


In [16]:
PosAggMetricRank[PosMetricList]

,IndCentRatio,MinABSSurvCoef,AvgABSSurvCoef,MinPosSigRate,AvgPosSigRate,MinABSGeCohD,AvgABSGeCohD
0,0.372093,2.133994,3.351779,0.19,0.3625,0.111486,0.177660
1,0.372093,2.133994,3.358863,0.22,0.4175,0.116090,0.187152
2,0.372093,2.133994,3.333053,0.37,0.4525,0.116775,0.190120
3,0.372093,2.133994,3.341116,0.26,0.4250,0.126021,0.192215
4,0.376093,2.146209,3.349088,0.26,0.4175,0.119659,0.184884
